# Вводные задачи

**1.1.** (уровень сложности: низкий)

Напишите функцию `deriv` для приближенного вычисления производной в заданной точке.

Пример работы:

```python
deriv(lambda x: x ** 3)(5)
75.00014999664018
```

In [1]:
def deriv(f, h=1e-6):
    def derivative_at_point(x):
        return (f(x + h) - f(x - h)) / (2 * h)

    return derivative_at_point


print(deriv(lambda x: x**3)(5))

75.0000000167006


**1.2.** (уровень сложности: средний)

Создайте вариант именованного кортежа с помощью ФВП. Классы и готовые структуры данных (словари, кортежи и так далее) использовать нельзя.

Примеры работы:

```python
p1 = person(name='Иван', age=20)
p2 = replace(replace(p1, 'name', 'Алексей'), 'age', 21)
get(p1, 'name'), get(p1, 'age')
# ('Иван', 20)
get(p2, 'name'), get(p2, 'age')
# ('Алексей', 21)
```

In [ ]:
def person(**attributes):
    def access(field):
        return attributes.get(field, None)

    return access


def replace(p, field, value):
    def new_access(f):
        if f == field:
            return value
        return p(f)

    return new_access


def get(p, field):
    return p(field)

In [2]:
# Примеры работы
p1 = person(name="Иван", age=20)
p2 = replace(replace(p1, "name", "Алексей"), "age", 21)

print(get(p1, "name"), get(p1, "age"))  # ('Иван', 20)
print(get(p2, "name"), get(p2, "age"))  # ('Алексей', 21)

Иван 20
Алексей 21


**1.3.** (уровень сложности: низкий)

Реализуйте рекурсивное вычисление факториала в виде выражения. Необходимо это сделать без использования именованных функций, переменных (в том числе без имени факториала) и присваиваний.

In [9]:
(lambda f: (lambda x: x(x))(lambda x: f(lambda v: x(x)(v))))(
    (lambda f: lambda n: 1 if n == 0 else n * f(n - 1))
)(5)

# лисп какой-то получился хз

120

In [10]:
(lambda f: (lambda x: x(x))(lambda x: f(lambda v: x(x)(v))))(
    (lambda f: lambda n: 1 if n == 0 else n * f(n - 1))
)(10)

3628800

**1.4.** (уровень сложности: низкий)

Создайте декоратор io, который задает функции для получения входных аргументов и вовращения результата.

Примеры:

```Python
@io(input, input, input, print)
def f1(x, y, z):
    return x + y + z

f1()
one
two
three
onetwothree
```

```Python
@io(lambda: random.random(), lambda: random.random(), lambda x: x)
def f2(x, y):
    return x * y

f2()
0.19896827110422532

In [ ]:
def io(*funcs):
    def decorator(func):
        def wrapper():
            args = [f() for f in funcs[:-1]]
            result = func(*args)
            funcs[-1](result)
            return result

        return wrapper

    return decorator

In [14]:
# Пример использования:
@io(lambda: input(), lambda: input(), lambda: input(), print)
def f1(x, y, z):
    return x + y + z


# Вызов функции f1
f1()

onetwothree


'onetwothree'

In [16]:
import random


@io(lambda: random.random(), lambda: random.random(), lambda x: x)
def f2(x, y):
    return x * y


f2()

0.06477916644564682

**1.5.** (уровень сложности: низкий)

Создайте декоратор класса @collect, который собирает все создаваемые объекты класса в единый список. К классу добавляется метод get_objects, который выдает этот список.

Пример:

```python
@collect
class C1:
    pass

a = C1()
b = C1()
c = C1()

C1.get_objects()
[<__main__.MyClass object at 0x00000259EDC10750>, <__main__.MyClass object at 0x00000259EDC968D0>, <__main__.MyClass object at 0x00000259EDC96910>]
```

In [18]:
def collect(cls):
    # список для хранения объектов
    cls._objects = []

    # копируем ориг конструктор
    original_init = cls.__init__

    # опред нового конструктора
    def new_init(self, *args, **kwargs):
        original_init(self, *args, **kwargs)  # вызов оригинального конструктора
        cls._objects.append(self)  # добавление созданный объект в список

    # редефайн конструктора
    cls.__init__ = new_init

    # Добавляем метод get_objects
    @classmethod
    def get_objects(cls):
        return cls._objects

    cls.get_objects = get_objects

    return cls

In [19]:
@collect
class C1:
    def __init__(self):
        pass


a = C1()
b = C1()
c = C1()

print(C1.get_objects())

[<__main__.C1 object at 0x119c00170>, <__main__.C1 object at 0x119c001a0>, <__main__.C1 object at 0x119c002c0>]


## 2. Работа со списками в функциональном стиле

Создайте тип данных односвязный список с помощью ФВП. При создании списка нельзя использовать классы, готовые списки, кортежи и так далее.

Добавьте ряд операций в функциональном стиле.

In [22]:
# создаем пустой список
def empty_list():
    return None


# проверка, пустой ли список
def is_empty(lst):
    return lst is None


# создание нового узла (элемента списка)
def cons(head, tail):
    return lambda f: f(head, tail)


# получение головы списка (первого элемента)
def head(lst):
    if is_empty(lst):
        raise ValueError("список пуст")
    return lst(lambda head, tail: head)


# получение хвоста списка (все элементы кроме первого)
def tail(lst):
    if is_empty(lst):
        raise ValueError("список пуст")
    return lst(lambda head, tail: tail)


# преобразование списка в строку для удобного вывода
def to_string(lst):
    def helper(lst, acc):
        if is_empty(lst):
            return acc
        else:
            return helper(tail(lst), acc + [head(lst)])

    return " -> ".join(map(str, helper(lst, [])))

In [23]:
# Пример использования
lst = empty_list()
lst = cons(3, lst)
lst = cons(2, lst)
lst = cons(1, lst)

print("Список:", to_string(lst))  # Вывод: 1 -> 2 -> 3
print("Первый элемент:", head(lst))  # Вывод: 1
print("Хвост:", to_string(tail(lst)))  # Вывод: 2 -> 3

Список: 1 -> 2 -> 3
Первый элемент: 1
Хвост: 2 -> 3


**2.1.** (уровень сложности: высокий)

Создайте функцию pair(head, tail), которая порождает элемент списка. Не используйте ветвления. Создайте также функции head(lst) (возвращает значение головы списка) и tail(lst) (возвращает хвост списка).

In [24]:
pair = cons

In [25]:
lst = pair(1, pair(2, pair(3, None)))

print("Голова:", head(lst))

print("Голова хвоста:", head(tail(lst)))

Голова: 1
Голова хвоста: 2


**2.2.** (уровень сложности: средний)

Создайте функцию make_list(`*args`), которая создает список на основе аргументов.

In [27]:
def make_list(*args):
    lst = None
    for element in reversed(args):
        lst = pair(element, lst)
    return lst


lst = make_list(1, 2, 3)

print("Голова:", head(lst))
print("Голова хвоста:", head(tail(lst)))
print("Список:", to_string(lst))

Голова: 1
Голова хвоста: 2
Список: 1 -> 2 -> 3


**2.3.** (уровень сложности: средний)

Создайте функцию list_to_string(lst), возвращающую строку, содержащую элементы списка.

In [28]:
list_to_string = to_string
print("Список:", list_to_string(lst))

Список: 1 -> 2 -> 3


**2.4.** (уровень сложности: средний)

Создайте функцию list_range(low, high), возвращающую список чисел от low до high включительно.

In [29]:
def list_range(low, high):
    if low > high:
        return None
    lst = None
    for i in reversed(range(low, high + 1)):
        lst = pair(i, lst)
    return lst

In [31]:
lst = list_range(1, 5)
print("Список:", to_string(lst))

Список: 1 -> 2 -> 3 -> 4 -> 5


**2.5.** (уровень сложности: средний)

Создайте функцию foldl(func, lst, acc), вычисляющую свертку элементов списка, аналогично reduce.

In [33]:
def foldl(func, lst, acc):
    if lst is None:
        return acc
    else:
        return foldl(func, tail(lst), func(acc, head(lst)))

In [34]:
lst = pair(1, pair(2, pair(3, pair(4, None))))


def add(x, y):
    return x + y


# применение foldl для суммирования элементов списка
result = foldl(add, lst, 0)
print("Сумма элементов списка:", result)

Сумма элементов списка: 10


**2.6.** (уровень сложности: средний)

Создайте функцию list_sum(lst) для вычисления суммы элементов списка с помощью foldl.

In [35]:
def list_sum(lst):
    return foldl(lambda acc, x: acc + x, lst, 0)


lst = pair(1, pair(2, pair(3, pair(4, None))))

# вычисление суммы элементов списка
result = list_sum(lst)
print("Сумма элементов списка:", result)  # Вывод: 10

Сумма элементов списка: 10


**2.7.** (уровень сложности: средний)

Создайте функцию fact(n) для вычисления факториала с помощью foldl и list_range.

In [36]:
def fact(n):
    if n < 0:
        raise ValueError("Факториал определен только для неотрицательных чисел")
    lst = list_range(1, n)
    return foldl(lambda acc, x: acc * x, lst, 1)

In [38]:
result = fact(5)
print("Факториал 5:", result)

print("Факториал 10:", fact(10))

Факториал 5: 120
Факториал 10: 3628800


**2.8.** (уровень сложности: средний)

Создайте функцию list_to_py(lst) для преобразования списка в обычный список Питона с помощью foldl.

In [40]:
def list_to_py(lst):
    return foldl(lambda acc, x: acc + [x], lst, [])

In [42]:
lst = pair(1, pair(2, pair(3, pair(4, None))))

# преобразование списка в обычный список Питона
py_list = list_to_py(lst)
print(
    f"Обычный список Питона: {py_list}\n{type(py_list) = }\n{py_list = }\n{type(lst) = }\n{lst = }"
)

Обычный список Питона: [1, 2, 3, 4]
type(py_list) = <class 'list'>
py_list = [1, 2, 3, 4]
type(lst) = <class 'function'>
lst = <function cons.<locals>.<lambda> at 0x119fae840>


**2.9.** (уровень сложности: средний)

Создайте функцию list_reverse(lst) для разворота списка в обратном направлении с помощью foldl.

In [43]:
def list_reverse(lst):
    return foldl(lambda acc, x: pair(x, acc), lst, None)

In [44]:
reversed_lst = list_reverse(lst)
print("Исходный список:", to_string(lst))
print("Развернутый список:", to_string(reversed_lst))

Исходный список: 1 -> 2 -> 3 -> 4
Развернутый список: 4 -> 3 -> 2 -> 1


**2.10.** (уровень сложности: средний)

Создайте функцию foldr(func, lst, acc), вычисляющую свертку справа для элементов списка.

In [45]:
def foldr(func, lst, acc):
    if lst is None:
        return acc
    else:
        return func(head(lst), foldr(func, tail(lst), acc))

In [46]:
result = foldr(add, lst, 0)
print("Сумма элементов списка (foldr):", result)

Сумма элементов списка (foldr): 10


**2.11.** (уровень сложности: средний)

Создайте функцию list_map(func, lst), аналог map, с помощью foldr.

In [47]:
def list_map(func, lst):
    return foldr(lambda x, acc: pair(func(x), acc), lst, None)

In [48]:
mapped_lst = list_map(lambda x: x * 2, lst)

print("Исходный список:", to_string(lst))
print("Преобразованный список:", to_string(mapped_lst))

Исходный список: 1 -> 2 -> 3 -> 4
Преобразованный список: 2 -> 4 -> 6 -> 8


**2.12.** (уровень сложности: средний)

Создайте функцию list_filter(pred, lst), аналог filter, с помощью foldr.

In [49]:
def list_filter(pred, lst):
    return foldr(lambda x, acc: pair(x, acc) if pred(x) else acc, lst, None)

In [50]:
lst = pair(1, pair(2, pair(3, pair(4, None))))
filtered_lst = list_filter(lambda x: x % 2 == 0, lst)

print("Исходный список:", to_string(lst))
print("Отфильтрованный список (четные числа):", to_string(filtered_lst))

Исходный список: 1 -> 2 -> 3 -> 4
Отфильтрованный список (четные числа): 2 -> 4


**2.13.** (уровень сложности: средний)

Создайте функцию sum_odd_squares для вычисления суммы квадратов нечетных чисел списка с помощью list_sum, list_map и list_filter.

In [51]:
def sum_odd_squares(lst):
    odd_lst = list_filter(lambda x: x % 2 != 0, lst)  # Фильтрация нечетных чисел
    squares_lst = list_map(lambda x: x * x, odd_lst)  # Вычисление квадратов
    return list_sum(squares_lst)  # Суммирование квадратов

In [52]:
lst = pair(1, pair(2, pair(3, pair(4, pair(5, None)))))

result = sum_odd_squares(lst)
print("Сумма квадратов нечетных чисел списка:", result)  # 35 (1^2 + 3^2 + 5^2)

Сумма квадратов нечетных чисел списка: 35


**2.14.** (уровень сложности: средний)

Создайте функцию list_concat(lst1, lst2) для соединения двух списков.

In [53]:
def list_concat(lst1, lst2):
    return foldr(pair, lst1, lst2)

In [54]:
lst1 = pair(1, pair(2, pair(3, None)))
lst2 = pair(4, pair(5, pair(6, None)))

concatenated_lst = list_concat(lst1, lst2)
print("Объединенный список:", to_string(concatenated_lst))

Объединенный список: 1 -> 2 -> 3 -> 4 -> 5 -> 6


**2.15.** (уровень сложности: средний)

Создайте функцию list_replace(lst, index, value) для изменения элемента списка по индексу.

In [55]:
def list_replace(lst, index, value):
    def replace_helper(lst, i):
        if lst is None:
            return None
        elif i == 0:
            return pair(value, tail(lst))
        else:
            return pair(head(lst), replace_helper(tail(lst), i - 1))

    return replace_helper(lst, index)

In [56]:
lst = pair(1, pair(2, pair(3, pair(4, None))))

replaced_lst = list_replace(lst, 2, 99)
print("Исходный список:", to_string(lst))
print("Измененный список:", to_string(replaced_lst))

Исходный список: 1 -> 2 -> 3 -> 4
Измененный список: 1 -> 2 -> 99 -> 4


## 6. Нормальные алгоритмы Маркова

**6.1.** (уровень сложности: средний)

Реализуйте интерпретатор [нормальных алгоритмов Маркова (НАМ)](https://www.mathnet.ru/php/archive.phtml?wshow=paper&jrnid=tm&paperid=1178&option_lang=rus). Обойдитесь без поддержки завершающих правил (стрелка с точкой).

Проверьте интерпретатор на алгоритме перевода из двоичной записи числа в унарное представление:

```python
rules = [
    ('|0', '0||'),
    ('1', '0|'),
    ('0', '')
]

print(markov('101', rules))
# |||||
```

In [61]:
def markov(s, rules):
    while True:
        for rule in rules:
            if rule[0] in s:
                s = s.replace(rule[0], rule[1])
                break
        else:
            break
    return s

In [62]:
rules = [("|0", "0||"), ("1", "0|"), ("0", "")]

print(markov("101", rules))

|||||


**6.2.** (уровень сложности: низкий)

Релизуйте НАМ для проверки четности двоичного числа. Пример работы:

```python
>>> markov('parity(11)', rules)
odd
>>> markov('parity(1010010)', rules)
even
```

In [64]:
rules = [
    ("parity(0)", "even"),
    ("parity(1)", "odd"),
    ("00", "0"),
    ("01", "1"),
    ("10", "0"),
    ("11", "1"),
    ("parity(", ""),
    (")", ""),
]

print(markov("parity(11)", rules))  # odd
print(markov("parity(1010010)", rules))  # even

odd
even


**6.3.** (уровень сложности: средний)

Реализуйте НАМ для проверки арифметического выражения на корректный синтаксис. Пример работы:

```python
>>> markov(' -12* (1 + 4) - (123 /3) ', rules)
E
>>> markov(' -12* (1 + 4+) - (123 /3) ', rules)
E*(E+)E
```

Если синтаксис корректен, возвращается `E`.

In [68]:
rules = [
    # Убираем все пробелы
    (" ", ""),
    # Убираем унарные операторы перед выражениями
    ("-E", "E"),
    ("+E", "E"),
    ("*E", "E"),
    ("/E", "E"),
    # Убираем бинарные операторы между выражениями
    ("E-E", "E"),
    ("E+E", "E"),
    ("E*E", "E"),
    ("E/E", "E"),
    # Убираем выражения в скобках
    ("(E)", "E"),
    # Убираем EE
    ("EE", "E"),
    # Убираем числа
    ("E0", "E"),
    ("E1", "E"),
    ("E2", "E"),
    ("E3", "E"),
    ("E4", "E"),
    ("E5", "E"),
    ("E6", "E"),
    ("E7", "E"),
    ("E8", "E"),
    ("E9", "E"),
    # Превращаем цифры в E
    ("0", "E"),
    ("1", "E"),
    ("2", "E"),
    ("3", "E"),
    ("4", "E"),
    ("5", "E"),
    ("6", "E"),
    ("7", "E"),
    ("8", "E"),
    ("9", "E"),
]

print(markov(" -12* (1 + 4) - (123 /3) ", rules))
print(markov(" -12* (1 + 4+) - (123 /3) ", rules))

E
E*(E+)E


**6.4.** (уровень сложности: хакер)

Расширьте НАМ до уровня языка переписывания термов (деревьев выражений): добавьте поддержку структурных скобок, метапеременных (в духе тех, что используются в конструкции match/case Питона) и функций.
